In [1]:
%cd ../

/u02/thanh/workplace/plasma


In [2]:
import time

import plasma.parallel_processing.communicators as ppc
import plasma.parallel_processing.queues as ppq
from plasma.functional import partials, AutoPipe, LambdaPipe

In [6]:
def compute(x):
    time.sleep(0.5)
    return f'compute: {x}'

def compute2(x):
    time.sleep(0.5)
    return f'compute2: {x + 'c'}'

class ComputeClass(AutoPipe):

    def __init__(self, hihi, haha):
        super().__init__()
    
    def run(self, tmp:int) -> str:
        return 'ComputeClass: ' + tmp + 'd'

In [4]:
def input_handler(i):
    return [i] * 2

In [7]:
out_queue= ppq.ThreadQueue().register_callback(print)
temp = partials(print, 'test')

class Flow(ppc.AsyncFlow):

    def __init__(self):
        super().__init__()

        distributor = ppc.distributors.IteratorDistributor()
        compute3 = ComputeClass(1, 2)
        self.chain(
            (ppq.ThreadQueue(), input_handler, distributor),
            (input_handler, compute, ppq.ThreadQueue()),
                (compute, print, ppq.ThreadQueue()),
                (compute, out_queue),
                (compute, compute2, ppq.ThreadQueue()),
                    (compute2, print),
                    (compute2, out_queue),
                    (compute2, compute3, ppq.ThreadQueue()),
                        (compute3, print),
                        (compute3, out_queue)
        )

flow = Flow()
flow

[ThreadQueue(name=None, runner=1, id=137134288888672)]

  |-><function input_handler at 0x7cb920196520>(id=137134549329184)-IteratorDistributor

    |->[ThreadQueue(name=None, runner=1, id=137134288704592)]

      |-><function compute at 0x7cb9109c04a0>(id=137134289454240)

        |->[ThreadQueue(name=None, runner=1, id=137134289174864)]

          |-><built-in function print>(id=137134637442096)

        |->[ThreadQueue(name=None, runner=1, id=137134288880784)]

        |->[ThreadQueue(name=None, runner=1, id=137134289521072)]

          |-><function compute2 at 0x7cb9109c0540>(id=137134289454400)

            |->[ThreadQueue(name=None, runner=1, id=137134289174864)]

              |-><built-in function print>(id=137134637442096)...

            |->[ThreadQueue(name=None, runner=1, id=137134288880784)]

            |->[ThreadQueue(name=None, runner=1, id=137134289522512)]

              |->ComputeClass[(tmp:int)->str](id=137134290664624)

                |->[ThreadQueue(name=None, ru

In [ ]:
with flow:
   for i in range(10):
      flow.put(i)

In [ ]:
flow.alive()

In [ ]:
out_queue.run()

In [ ]:
out_queue.release()